### En este Notebook definimos el alcance del proyecto eligiendo que categorias vamos a usar, para llegar a la conclusion de las categorias puede revisar el Notebook EDA/eda.ipynb donde con procesos de ML se normalizaron las categorias para poder filtrar aqui el alcance por estados y categorias mas relevantes del dataset.

In [2]:
import pandas as pd

In [2]:
# Leemos el archivo de google que tenemos hasta el momento.
g_sitios = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\google_sitios_v1.parquet', engine='pyarrow')
g_sitios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2888854 entries, 0 to 2888853
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   name            object 
 1   id_negocio      object 
 2   description     object 
 3   avg_rating      float64
 4   num_of_reviews  int64  
 5   price           object 
 6   url             object 
 7   id_estado       int32  
dtypes: float64(1), int32(1), int64(1), object(5)
memory usage: 165.3+ MB


In [3]:
g_sitios.head()

name                             id_negocio description  \
0   Porter Pharmacy  0x88f16e41928ff687:0x883dad4fd048e8f8        None   
1      City Textile  0x80c2c98c0e3c16fd:0x29ec8a728764fdf9        None   
2      San Soo Dang  0x80c2c778e3b73d33:0xbdc58662a4a97d49        None   
3      Nova Fabrics   0x80c2c89923b27a41:0x32041559418d447        None   
4  Nobel Textile Co  0x80c2c632f933b073:0xc31785961fe826a6        None   

   avg_rating  num_of_reviews price  \
0         4.9              16  None   
1         4.5               6  None   
2         4.4              18  None   
3         3.3               6  None   
4         4.3               7  None   

                                                 url  id_estado  
0  https://www.google.com/maps/place//data=!4m2!3...          1  
1  https://www.google.com/maps/place//data=!4m2!3...          2  
2  https://www.google.com/maps/place//data=!4m2!3...          2  
3  https://www.google.com/maps/place//data=!4m2!3...          2  
4  https://www.google.com/maps/place//data=!4m2!3...          2

In [4]:
# Abrimos el archivo de categorias finales que se proceso en el EDA.
categorias_g = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\google_categories_final.parquet', engine='pyarrow')
categorias_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6211324 entries, 0 to 6211323
Data columns (total 5 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   gmap_id              object
 1   category_id_google   int64 
 2   category             object
 3   category_general     object
 4   id_category_general  int64 
dtypes: int64(2), object(3)
memory usage: 236.9+ MB


In [5]:
# Renombramos las columna para mejor entendimiento de la situacion en el futuro
categorias_g.rename(columns={'gmap_id': 'id_negocio'}, inplace=True)
# Hacemos una copia antes de editar el dataset por si necesitamos la informacion antigua.
todas_categorias = categorias_g.copy()
# Agrupamos las categorias generales por el id_negocio existente en la tabla de google
categorias_g = categorias_g.groupby('id_negocio').agg({'id_category_general': 'first'}).reset_index()
categorias_g.head()

id_negocio  id_category_general
0  0x0:0x117c7fa82238f16a                    3
1  0x0:0x13154eabf70b4b93                    8
2  0x0:0x1cda8cf8c096e582                    5
3  0x0:0x2522ed462fd64085                    5
4  0x0:0x2ab217fc59211262                    5

In [6]:
# Mezclamos esta informacion para que la tabla de google ahora tenga un id de categoria
g_sitios= pd.merge(g_sitios, categorias_g[['id_negocio', 'id_category_general']], on='id_negocio', how='left')
g_sitios


name  \
0                       Porter Pharmacy   
1                          City Textile   
2                          San Soo Dang   
3                          Nova Fabrics   
4                      Nobel Textile Co   
...                                 ...   
2888849                        Steak 48   
2888850   Jack Mcnerney Chevrolet, Inc.   
2888851  Central Ny Spay Neuter Assista   
2888852                   Ok Feed Store   
2888853              Crestview Crossing   

                                    id_negocio description  avg_rating  \
0        0x88f16e41928ff687:0x883dad4fd048e8f8        None         4.9   
1        0x80c2c98c0e3c16fd:0x29ec8a728764fdf9        None         4.5   
2        0x80c2c778e3b73d33:0xbdc58662a4a97d49        None         4.4   
3         0x80c2c89923b27a41:0x32041559418d447        None         3.3   
4        0x80c2c632f933b073:0xc31785961fe826a6        None         4.3   
...                                        ...         ...         ...   
2888849  0x89c6c74f43a49b55:0x6be6995921c58b12        None         4.6   
2888850  0x89da1787d3fdc4a5:0xce92ea6e3cd4d1cc        None         4.4   
2888851  0x89da6b9ce832726f:0xbd0d3630f1e3b42e        None         4.3   
2888852  0x88d9e872a9e39f3d:0x5de29f50507e5d73        None         4.9   
2888853  0x54956b3944b1654d:0x7797cead9a485c22        None         5.0   

         num_of_reviews price  \
0                    16  None   
1                     6  None   
2                    18  None   
3                     6  None   
4                     7  None   
...                 ...   ...   
2888849             308  None   
2888850             143  None   
2888851              88  None   
2888852              13  None   
2888853               1  None   

                                                       url  id_estado  \
0        https://www.google.com/maps/place//data=!4m2!3...          1   
1        https://www.google.com/maps/place//data=!4m2!3...          2   
2        https://www.google.com/maps/place//data=!4m2!3...          2   
3        https://www.google.com/maps/place//data=!4m2!3...          2   
4        https://www.google.com/maps/place//data=!4m2!3...          2   
...                                                    ...        ...   
2888849  https://www.google.com/maps/place//data=!4m2!3...         12   
2888850  https://www.google.com/maps/place//data=!4m2!3...         13   
2888851  https://www.google.com/maps/place//data=!4m2!3...         13   
2888852  https://www.google.com/maps/place//data=!4m2!3...          5   
2888853  https://www.google.com/maps/place//data=!4m2!3...         44   

         id_category_general  
0                        5.0  
1                        5.0  
2                        5.0  
3                        5.0  
4                        5.0  
...                      ...  
2888849                  3.0  
2888850                  4.0  
2888851                  4.0  
2888852                  4.0  
2888853                  5.0  

[2888854 rows x 9 columns]

In [7]:
# Filtramos el dataset para que solo queden las categorias de Restaurantes = 14. y Hoteles = 7
google_filtrado = [14, 7]
g_sitios = g_sitios[g_sitios['id_category_general'].isin(google_filtrado)]
g_sitios

name  \
413            Seligman Fountain - "Bear and Faun"   
509                                   101 Mobility   
539                     Bayrunner Fishing Charters   
672                              JLM Manufacturing   
1239                       All Time Auto Transport   
...                                            ...   
2888708                           Arro Corporation   
2888736  Hoquiam Licensing & Transportation Agency   
2888757                        Fountain Hills Pool   
2888759                               Tanyard Cove   
2888845                                  Fuel-N-Go   

                                    id_negocio description  avg_rating  \
413      0x89c2f6163ac2864f:0xa4213a2f66c15d8f        None         4.5   
509       0x864c1f10c079a921:0x95710cd26231cc3        None         5.0   
539      0x88fc79ee1ededc73:0x7003065a75ee0cf4        None         4.8   
672      0x8824d759c04cb37b:0xef1cb7398b3d692e        None         3.7   
1239     0x88d9050582e1e64b:0x33457ba08c64df20        None         4.8   
...                                        ...         ...         ...   
2888708  0x880e49ad2e2a84fd:0x19e678ebd62178fc        None         3.6   
2888736  0x54923990302cb3c7:0x10f023b56efdb019        None         4.3   
2888757  0x89b62c2db5856cd3:0x89e0d1a12b77f78a        None         4.4   
2888759  0x89b7fdc26ba070ef:0x376e99b8fe4bfc32        None         4.5   
2888845  0x54c0e365e43253f9:0x4366520651be08bb        None         4.2   

         num_of_reviews price  \
413                   4  None   
509                  45  None   
539                  34  None   
672                  33  None   
1239                 96  None   
...                 ...   ...   
2888708               5  None   
2888736              94  None   
2888757              18  None   
2888759              76  None   
2888845              16  None   

                                                       url  id_estado  \
413      https://www.google.com/maps/place//data=!4m2!3...         13   
509      https://www.google.com/maps/place//data=!4m2!3...          6   
539      https://www.google.com/maps/place//data=!4m2!3...         28   
672      https://www.google.com/maps/place//data=!4m2!3...         10   
1239     https://www.google.com/maps/place//data=!4m2!3...          5   
...                                                    ...        ...   
2888708  https://www.google.com/maps/place//data=!4m2!3...         22   
2888736  https://www.google.com/maps/place//data=!4m2!3...         16   
2888757  https://www.google.com/maps/place//data=!4m2!3...         24   
2888759  https://www.google.com/maps/place//data=!4m2!3...         24   
2888845  https://www.google.com/maps/place//data=!4m2!3...         44   

         id_category_general  
413                     14.0  
509                      7.0  
539                      7.0  
672                     14.0  
1239                    14.0  
...                      ...  
2888708                 14.0  
2888736                  7.0  
2888757                  7.0  
2888759                  7.0  
2888845                  7.0  

[46524 rows x 9 columns]

In [8]:
# Terminado la informacion. guardamos el dataframe principal actualizado en parquet.
import os
os.makedirs('../../../data', exist_ok=True)
g_sitios.to_parquet('../../../data/google_sitios_v2.parquet', index=False)

In [9]:
# Contamos la cantidad de restaurantes y hoteles que contienen los estados, para elegir un Top 5 de mayor cantidad de negocios 
conteo_negocios_por_estado = g_sitios.groupby('id_estado').size().reset_index(name='cantidad_negocios')
conteo_negocios_por_estado.sort_values(by='cantidad_negocios', ascending=False)

id_estado  cantidad_negocios
1           2               5140
4           5               3715
5           6               3576
12         13               2993
11         12               1952
13         14               1566
21         22               1419
19         20               1360
9          10               1341
0           1               1323
15         16               1313
42         43               1210
29         30               1094
31         32               1032
24         25               1003
14         15                964
34         35                938
28         29                887
10         11                842
43         44                833
20         21                786
38         39                752
27         28                694
23         24                675
37         38                643
17         18                617
6           7                611
16         17                591
32         33                551
8           9                518
39         40                486
2           3                473
22         23                437
36         37                421
26         27                349
46         47                334
3           4                313
40         41                307
47         48                278
35         36                278
33         34                276
25         26                267
18         19                226
7           8                222
30         31                167
45         46                165
49         50                152
41         42                141
44         45                140
48         49                138
50         51                 15

In [10]:
# Calcular el total de negocios
total_negocios = conteo_negocios_por_estado['cantidad_negocios'].sum()

# Calcular el porcentaje del total para cada id_estado
conteo_negocios_por_estado['porcentaje'] = round((conteo_negocios_por_estado['cantidad_negocios'] / total_negocios) * 100,2)
exportar_info = conteo_negocios_por_estado.copy() 
# Mostrar el resultado
conteo_negocios_por_estado = conteo_negocios_por_estado.sort_values(by='porcentaje', ascending=False).head(5)

- 2 = California
- 5 = Florida
- 6 = Texas
- 13 = New York
- 12 = Pennsylvania

Ahora que achicamos la informacion en la tabla principal. tambien debemos achicar la informacion de las demas tablas (address, horarios, misc, relative_results). 

In [6]:
# Leemos el archivo de google que tenemos hasta el momento.
g_sitios = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\google_sitios_v2.parquet', engine='pyarrow')
google_estados = [2,5,6,13,12]
g_sitios = g_sitios[g_sitios['id_estado'].isin(google_estados)]
g_sitios.reset_index()
g_sitios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17376 entries, 0 to 46518
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 17376 non-null  object 
 1   id_negocio           17376 non-null  object 
 2   description          1246 non-null   object 
 3   avg_rating           17376 non-null  float64
 4   num_of_reviews       17376 non-null  int64  
 5   price                954 non-null    object 
 6   url                  17376 non-null  object 
 7   id_estado            17376 non-null  int32  
 8   id_category_general  17376 non-null  float64
dtypes: float64(2), int32(1), int64(1), object(5)
memory usage: 1.3+ MB


In [7]:
# Leemos los archivos que tenemos hasta el momento.
g_horarios = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\g_horarios_v2.parquet', engine='pyarrow')
g_misc = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\g_misc_v3.parquet', engine='pyarrow')
g_address = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\g_address_v2.parquet', engine='pyarrow')
g_relative_results = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\g_relative_results_v2.parquet', engine='pyarrow')

In [ ]:
# Renombramos los id para mayor intuitividad. 
g_relative_results.rename(columns={'gmap_id': 'id_negocio'}, inplace=True)
g_misc.rename(columns={'gmap_id': 'id_negocio'}, inplace=True)

In [8]:
# Obtenemos los ID unicos
g_sitios_ids = g_sitios['id_negocio'].unique()

# Filtrar las tablas por los IDs que coincidan con los de g_sitios
address = g_address[g_address['id_negocio'].isin(g_sitios_ids)]
horarios = g_horarios[g_horarios['id_negocio'].isin(g_sitios_ids)]
misc = g_misc[g_misc['id_negocio'].isin(g_sitios_ids)]
relative_r = g_relative_results[g_relative_results['id_negocio'].isin(g_sitios_ids)]

# Verificacion
print(f"Tamaño de la tabla address filtrada: {len(address)} filas")
print(f"Tamaño de la tabla horarios filtrada: {len(horarios)} filas")
print(f"Tamaño de la tabla horarios filtrada: {len(misc)} filas")
print(f"Tamaño de la tabla horarios filtrada: {len(relative_r)} filas")

Tamaño de la tabla address filtrada: 17376 filas
Tamaño de la tabla horarios filtrada: 17376 filas
Tamaño de la tabla horarios filtrada: 53333 filas
Tamaño de la tabla horarios filtrada: 69812 filas


In [10]:
# Guardamos las tablas nuevas actualizadas.
import os
os.makedirs('../../../data', exist_ok=True)
address.to_parquet('../../../data/g_address_v3.parquet', index=False)
horarios.to_parquet('../../../data/g_horarios_v3.parquet', index=False)
misc.to_parquet('../../../data/g_misc_v4.parquet', index=False)
relative_r.to_parquet('../../../data/g_relative_results_v3.parquet', index=False)
g_sitios.to_parquet('../../../data/g_sitios_v3.parquet', index=False)